<a href="https://colab.research.google.com/github/Daudi232/DS_1/blob/main/NLP_Hugging_Face.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## WEEK 6:1

In [ ]:
## QUIZ
# demonstration of grouped entities

from transformers import pipeline

ner = pipeline("ner", grouped_entities=True)

ner("My name is Sylvain and I work at Hugging Face in Brooklyn.")

In [ ]:
ner = pipeline ("ner", grouped_entities = True)

# grouped entities groups the entities mentioned in the text.. eg Anna Sandor as a person

ner ("Her name is Anna Sandor. She is a masters student in Sports Economics. She is cute, but currently sick, Hope she gets better soon!")

In [ ]:
#<mask> tag is different depending with model you are using
fill_blank = pipeline ("fill-mask", model = 'distilroberta-base')

fill_blank ("Her name is Anna, she is so <mask>")

In [ ]:
# you should specify candidate labels as an input
# the model did not have the labels specified here during its training

classifier = pipeline("zero-shot-classification")
result = classifier("This is a course about the Transformers in NLP", candidate_labels = ['education', 'sports', 'movies'])
print (result)

## WEEK 6: 2

In [ ]:
#!pip install datasets evaluate transformers[sentencepiece]

In [ ]:
# Sentenses to use in classification

s1 = 'My name is Daudi. I kind of like Manchester united.. only when they win'
s2 = 'My relationship with Irene is a rollercoaster, sometimes love, sometimes hate, sometimes joy, sometimes anger'
s3 = 'My students are funny, one minute they are engaged, another they are bored. I have a good time with them though'

In [ ]:
# SA using pipeline

from transformers import pipeline

classifier = pipeline("sentiment-analysis")

classifier(
    [s1, s2, s3]
)

In [ ]:
# We want to re-do the above following the process.. a little bit low level

## TOKENIZATION

from transformers import AutoTokenizer

checkpoint = 'distilbert-base-uncased-finetuned-sst-2-english' # get the model name
tokenizer = AutoTokenizer.from_pretrained (checkpoint) # get the tokenizer

raw = [s1, s2, s3]

# pass in your sentenses for tokenization and conversion to input ids
# I have not yet looked at what padding and truncation do
# I did not experiment on the effect it has if you leave them as false

output_tokenizer = tokenizer (raw, padding = True, truncation = True, return_tensors='tf')
print (output_tokenizer)

In [ ]:
print (output_tokenizer['attention_mask'])
print(output_tokenizer['input_ids'])


In [ ]:
## EMBEDDING PART using TFAutoModel

from transformers import TFAutoModel

model = TFAutoModel.from_pretrained (checkpoint)
vec_embs = model (output_tokenizer) #vector embeddings
print (vec_embs)

In [ ]:
# different ways of getting the vector embeddings

vec_embs ['last_hidden_state']

vec_embs [0]

vec_embs.last_hidden_state

In [ ]:
## Using the model-head specific for a certain task and bypassing the embedding step

from transformers import TFAutoModelForSequenceClassification #a mouthful

model_head = TFAutoModelForSequenceClassification.from_pretrained(checkpoint)
model_head_output = model_head (output_tokenizer)

print (model_head_output)

In [ ]:
type (model_head_output) #TFSequenceClassifier

In [ ]:
# these are like the log odds-ratio between the 2 classes
# looks like the output layer of the model-head is just linear
# remember the logistic assumption: that logit = hypothesis (dot-product of input and weights)

model_head_output['logits']

model_head_output.logits
#weird why this does not work.. thought it may considering it is part of the output
#model_head_output.dtype

In [ ]:
import tensorflow as tf

In [ ]:
# Post-Processing
# converting the logits to probabilities

model_sentiment_scores = tf.math.softmax (model_head_output.logits)
model_sentiment_scores

In [ ]:
# Checking the configuration of the model
# shows the configuration of the model

model.config

In [ ]:
# checking out labels

model.config.id2label

In [ ]:
# negative sentiment scores for the sentences

# first word was considered negative
model_sentiment_scores[:,0]

In [ ]:
# positive sentiment scores for the sentences
model_sentiment_scores[:,1]

In [ ]:
from transformers import BertConfig, TFBertModel

# Taking a look at the config that was used by bert during its training
config_bert = BertConfig()

# Building a model from scratch using the bert-configuration
model_bert = TFBertModel(config_bert)

In [ ]:
## INSIDE THE MODEL

from transformers import GPT2Config, TFGPT2Model

# Building the configuration

config_gpt = GPT2Config()

# Building a model from config
model_gpt = TFGPT2Model (config_gpt)

print (config_gpt)
# even the names of the configuration parameters is not identical
print (config_bert)

In [ ]:
# I have got no way of measuring the embedding vectors that have been generated here but they should be shit since the weights
# used here are just randomly initialized
# these weights are to be used if we want to pre-train this model from scratch..

model_gpt(output_tokenizer)

In [ ]:
# Instead of using the random weights, we can use the pre-trained weights

model_gpt_pt = TFGPT2Model.from_pretrained ('gpt2')
model_gpt_pt(output_tokenizer) # gives us the embeddings of the input ids that were generated by AutoTokenizer

# Alternative to the TFGPT2Model.. this is more general
# remember this is just for the embeddings, not a specific task
model_gpt_auto = TFAutoModel.from_pretrained ('gpt2')

In [ ]:
# Saving the model parameters

model_gpt_pt.save_pretrained("GPT2_model")

In [ ]:
## Re-doing the above.. this time using simple words

import numpy as np
sequences = ["Hello!", "Cool.", "Nice!"]

encoded_sequences = np.array ([
    [101, 7592, 999, 102],
    [101, 4658, 1012, 102],
    [101, 3835, 999, 102],
])

encoded_sequences_tensor = tf.constant (encoded_sequences)

In [ ]:
# using a list as an input for the model does not work
# using a numpy array or tensor does work
model_gpt_pt (encoded_sequences)

## WEEK 7

In [ ]:
# We want to re-do the above following the process.. a little bit low level

## TOKENIZATION

from transformers import AutoTokenizer

checkpoint = 'distilbert-base-uncased-finetuned-sst-2-english' # get the model name
tokenizer = AutoTokenizer.from_pretrained (checkpoint) # get the tokenizer

raw = [s1, s2, s3]

# pass in your sentenses for tokenization and conversion to input ids
# I have not yet looked at what padding and truncation do
# I did not experiment on the effect it has if you leave them as false

output_tokenizer = tokenizer (raw, padding = True, truncation = True, return_tensors='tf')
print (output_tokenizer)

In [ ]:
from transformers import AutoTokenizer #i use transformer a lot instead of transformers

tokenizer = AutoTokenizer.from_pretrained ('distilbert-base-uncased-finetuned-sst-2-english')

# Sentenses to use in tokenization
s1 = 'My name is Daudi. I kind of like Manchester united.. only when they win!'
s2 = 'My relationship with Irene is a rollercoaster, sometimes love, sometimes hate, sometimes joy, sometimes anger'
s3 = 'My students are funny, one minute they are engaged, another they are bored. I have a good time with them though!'

# looks like tokenizing many sequences at the same time is not applicable here

raw = (s1,s2,s3)
tokenized_sentense = tokenizer.tokenize (s1)

print (tokenized_sentense)

In [ ]:
# Tokenization using different models

tokenizer = AutoTokenizer.from_pretrained ('distilbert-base-uncased-finetuned-sst-2-english')
tokenizer2 = AutoTokenizer.from_pretrained ("bert-base-cased") # same tokenization as distilbert.. ## used
tokenizer3 = AutoTokenizer.from_pretrained ("gpt2") ##G used in middle words

tokenizer2.tokenize(s1)
tokenizer3.tokenize(s1)

In [ ]:
tokenizer(s1)

In [ ]:
tokenizer([s1,s2])

In [ ]:
# Encoding tokens

encoded_sequence = tokenizer3.convert_tokens_to_ids(tokenizer3.tokenize(s1))
print (encoded_sequence)

In [ ]:
# Decoding tokens

decoded_sequence = tokenizer3.decode (encoded_sequence)
print (decoded_sequence)

In [ ]:
import tensorflow as tf
enc_seq_tensor = tf.constant (encoded_sequence)

In [ ]:
enc_seq_tensor

In [ ]:
from transformers import TFAutoModel, TFAutoModelForSequenceClassification

model = TFAutoModel.from_pretrained('gpt2') #the number of times i forget .from_pretrained is insane
model_head = TFAutoModelForSequenceClassification.from_pretrained ('gpt2')

emb_vec = model(enc_seq_tensor) #embeddings are generated
model_head_output = model_head(enc_seq_tensor) #

print (model_head_output) # i expected logits as to whether the sequence is +ve or -ve but got embeddings instead

In [ ]:
#gpt_tokenizer = tokenizer3(s1, padding = True, truncation = True, return_tensors = 'tf')
gpt_tokenizer = tokenizer3(s1, truncation = True, return_tensors = 'tf')
model_head_output_2 = model_head(gpt_tokenizer)

In [ ]:
model_head_output_2

In [ ]:
import tensorflow as tf
from transformers import AutoTokenizer, TFAutoModelForSequenceClassification

checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = TFAutoModelForSequenceClassification.from_pretrained(checkpoint)

sequence = "I've been waiting for a HuggingFace course my whole life."

tokens = tokenizer.tokenize(sequence)
ids = tokenizer.convert_tokens_to_ids(tokens)
input_ids = tf.constant(ids) # it was a 1 rank-tensor
# This line will fail. #actually it has worked pretty well
print(model(input_ids))

In [ ]:
# This is the point of the next cell, but it is not necessary
print (tf.constant([1,2,3]))
print (tf.constant([[1,2,3]]))

In [ ]:
import tensorflow as tf
from transformers import AutoTokenizer, TFAutoModelForSequenceClassification

checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = TFAutoModelForSequenceClassification.from_pretrained(checkpoint)

sequence = "I've been waiting for a HuggingFace course my whole life."

tokens = tokenizer.tokenize(sequence)
ids = tokenizer.convert_tokens_to_ids(tokens)
input_ids = tf.constant([ids]) #2 ranked tensor
# This line will fail. #actually it has worked pretty well
print(model(input_ids))

In [ ]:
## Using gpt2 for classification like this won't work

import tensorflow as tf
from transformers import AutoTokenizer, TFAutoModelForSequenceClassification

#checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
tokenizer = AutoTokenizer.from_pretrained('gpt2')
model = TFAutoModelForSequenceClassification.from_pretrained('gpt2')

sequence = "I've been waiting for a HuggingFace course my whole life."

tokens = tokenizer.tokenize(sequence)
ids = tokenizer.convert_tokens_to_ids(tokens)
input_ids = tf.constant(ids)
print(model(input_ids)) # using gpt2 will give out embeddings instead of logits

In [ ]:
## This should have been how gpt2 could be used, but the results are gibberish

# from transformers import TFGPT2LMHeadModel, GPT2Tokenizer

# tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
# model = TFGPT2LMHeadModel.from_pretrained('gpt2')

# inputs = tokenizer(sequence, return_tensors='tf')
# outputs = model.generate(**inputs, max_length=50)
# print(tokenizer.decode(outputs[0]))

In [ ]:
## Batching sequences of different lengths

model = TFAutoModelForSequenceClassification.from_pretrained(checkpoint)
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

sequence1_ids = [[200, 200, 200]]
sequence2_ids = [[200, 200]]

pad_token_id = 100

batched_ids = [
    [200, 200, 200],
    [200, 200, pad_token_id]
]

print (model(tf.constant(sequence1_ids)))
print (model(tf.constant(sequence2_ids)))

print (model(tf.constant(batched_ids)))

In [ ]:
## Batching sequences of different lengths

model = TFAutoModelForSequenceClassification.from_pretrained(checkpoint)
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
# tokenizer.pad_token_id

sequence1_ids = [[200, 200, 200]]
sequence2_ids = [[200, 200]]

pad_token_id = 100
# tokenizer.pad_token_id
# the above is the pad_token_id that is used.. it is the number 0 actually!

batched_ids = [
    [200, 200, 200],
    [200, 200, pad_token_id]
]

# attention mask helps in ignoring the padded ids that are helpful in matching lengths of sequences
attention_mask = [[1, 1, 1],
                  [1, 1, 0]
                  ]

print (model(tf.constant(sequence1_ids)))
print (model(tf.constant(sequence2_ids)))

# attention mask enabled
print (model(tf.constant(batched_ids), attention_mask = tf.constant(attention_mask)))


In [ ]:
## PUTTING IT ALL TOGETHER

from transformers import AutoTokenizer

checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

sequence = "I've been waiting for a HuggingFace course my whole life."

model_inputs = tokenizer(sequence)
print (model_inputs)

In [ ]:
sequences = ["I've been waiting for a HuggingFace course my whole life.", "So have I!"]

model_inputs = tokenizer(sequences)
print (model_inputs)

# if we try running this code, it will complain because the model inputs have not been padded
#model (model_inputs)

In [ ]:
# Will pad the sequences up to the maximum sequence length

## PADDING

model_inputs = tokenizer(sequences, padding="longest")
print (model_inputs)
print (len(model_inputs['input_ids'][0]))
print (len(model_inputs['attention_mask'][0]))

In [ ]:
# Will pad the sequences up to the model max length
# (512 for BERT or DistilBERT)
model_inputs = tokenizer(sequences, padding="max_length")
print (model_inputs)
print (len(model_inputs['input_ids'][0]))
print (len(model_inputs['attention_mask'][0]))

In [ ]:
# Will pad the sequences up to the specified max length
model_inputs = tokenizer(sequences, padding="max_length", max_length=8)
print (model_inputs)
print (len(model_inputs['input_ids'][0]))
print (len(model_inputs['attention_mask'][0]))

In [ ]:
## TRUNCATION
# Will truncate the sequences that are longer than the model max length
# (512 for BERT or DistilBERT)
model_inputs = tokenizer(sequences, truncation=True)
print (model_inputs)

In [ ]:
# Will truncate the sequences that are longer than the specified max length
model_inputs = tokenizer(sequences, max_length=8, truncation=True)
print (model_inputs)

In [ ]:
## RETURN TENSORS

# Will truncate the sequences that are longer than the specified max length
model_inputs = tokenizer(sequences, max_length=8, truncation=True, padding = True, return_tensors = 'tf')
print (model_inputs)

In [ ]:
## RETURN TENSORS

# Will truncate the sequences that are longer than the specified max length
model_inputs = tokenizer(sequences, truncation=True, padding = 'longest', return_tensors = 'pt')
print (model_inputs)

In [ ]:
model_inputs = tokenizer(sequences, padding=True, return_tensors="np")
print (model_inputs)

In [ ]:
sequence = "I've been waiting for a HuggingFace course my whole life."

model_inputs = tokenizer(sequence)
print(model_inputs["input_ids"])

tokens = tokenizer.tokenize(sequence)
ids = tokenizer.convert_tokens_to_ids(tokens)
print(ids) #there is an extra ID added at the starting and end position when we use tokenizer

print (tokenizer.decode(model_inputs["input_ids"]))

print (print (tokenizer.decode(ids)))

In [ ]:
## Wrapping it up

import tensorflow as tf
from transformers import AutoTokenizer, TFAutoModelForSequenceClassification

checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = TFAutoModelForSequenceClassification.from_pretrained(checkpoint)
sequences = ["I've been waiting for a HuggingFace course my whole life.", "So have I!"]

tokens = tokenizer(sequences, padding=True, truncation=True, return_tensors="tf")
output = model(**tokens)
print (output)

In [ ]:
## QUIZ

from transformers import AutoTokenizer, TFAutoModel # When using TF.. TF before AutoModel is critical!

checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
seq = 'My time is limited, there is a lot of things to do'

tokenizer = AutoTokenizer.from_pretrained(checkpoint)
input_model = tokenizer (seq, return_tensors = 'tf')

model = TFAutoModel.from_pretrained (checkpoint)
output_model = model (input_model)

print (output_model)

In [ ]:
# SUGGESTION FROM DEEPSEEK

# from transformers import AutoTokenizer, TFAutoModel  # Note: Using TFAutoModel for TensorFlow
# import tensorflow as tf

# checkpoint = "bert-base-uncased"  # Explicitly define the checkpoint
# seq = 'My time is limited, there is a lot of things to do'

# # Initialize tokenizer and model
# tokenizer = AutoTokenizer.from_pretrained(checkpoint)
# model = TFAutoModel.from_pretrained(checkpoint)  # Using TensorFlow variant

# # Tokenize input (returns TensorFlow tensors)
# inputs = tokenizer(seq, return_tensors="tf")

# # Get model outputs
# outputs = model(**inputs)

# # Print outputs
# print("Last hidden states shape:", outputs.last_hidden_state.shape)
# print("Pooler output shape:", outputs.pooler_output.shape)

## WEEK 8
Fine tuning a pre-trained model

In [ ]:
# import tensorflow as tf
# import numpy as np
# from transformers import AutoTokenizer, TFAutoModelForSequenceClassification

# # Same as before
# checkpoint = "bert-base-uncased"
# tokenizer = AutoTokenizer.from_pretrained(checkpoint)
# model = TFAutoModelForSequenceClassification.from_pretrained(checkpoint)
# sequences = [
#     "I've been waiting for a HuggingFace course my whole life.",
#     "This course is amazing!",
# ]

# batch = dict(tokenizer(sequences, padding=True, truncation=True, return_tensors="tf"))

# # This is new
# model.compile(optimizer="adam", loss="sparse_categorical_crossentropy")
# labels = tf.convert_to_tensor([1, 1])
# model.train_on_batch(batch, labels)

In [ ]:
# I changed line 14 and 18 of the previous code
# This part sort of introduces fine-tuning a model from hugging face using own
# data.. this data has just 2 sentenses so i presume it will be shit!

import tensorflow as tf
import numpy as np
from transformers import AutoTokenizer, TFAutoModelForSequenceClassification

# Same as before
checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = TFAutoModelForSequenceClassification.from_pretrained(checkpoint)
sequences = [
    "I've been waiting for a HuggingFace course my whole life.",
    "This course is amazing!",
]

batch = tokenizer(sequences, padding=True, truncation=True, return_tensors="tf")

# This is new
model.compile(optimizer="adam", loss="sparse_categorical_crossentropy")

labels = tf.constant([1, 1])
model.train_on_batch(batch, labels)
model.summary()

In [ ]:
# Predict on new text
new_sequences = ["I hate her", "Today was horrible."]
new_batch = tokenizer(new_sequences, padding=True, truncation=True, return_tensors="tf")
predictions = model.predict(new_batch)
print(predictions.logits)  # Raw output scores

In [ ]:
probabilities = tf.math.softmax(predictions.logits, axis=-1)
print(probabilities)  # Class probabilities

In [ ]:
print(model.config)

In [ ]:
model.config.id2label

In [ ]:
!pip install datasets

In [ ]:
from datasets import load_dataset
import pandas as pd

raw_datasets = load_dataset("glue", "mrpc")
df = pd.DataFrame (raw_datasets ['train']) # train to dataframe for better view

In [ ]:
df.iloc[0,:]

In [ ]:
print (df.iloc[0,0])
print (df.iloc[0,1])
print (df.iloc[1,0])
print (df.iloc[1,1])

In [ ]:
print (type (raw_datasets)) #not a dictionary.. it is a dataset object
print (raw_datasets)

In [ ]:
# I guess this is a better view
print (raw_datasets['train'][0])
print (raw_datasets['train'][1])

In [ ]:
# This is super weird for now
raw_datasets['train'].features

In [ ]:
# after the train access.. the dataset object sort of becomes like a dataframe, where the features are accessed by their names
print (raw_datasets['train']['sentence1'][0:3])
print (raw_datasets['train']['sentence2'][0:3])
print (raw_datasets['train']['label'][0:3])

In [ ]:
# sentense-pair 15
raw_datasets ['train'][15]

In [ ]:
raw_datasets ['train']['sentence1']

In [ ]:
## TOKENIZATION


from transformers import AutoTokenizer

checkpoint = 'bert-base-uncased'
tokenizer = AutoTokenizer.from_pretrained (checkpoint)

# This approach is not proper, since we need the sentences to be used as a pair, not separately
sentence1_tokenizer = tokenizer (raw_datasets['train']['sentence1'], padding = True, truncation = True, return_tensors = 'tf')
sentence1_tokenizer = tokenizer (raw_datasets['train']['sentence1'], padding = True, truncation = True, return_tensors = 'tf')

In [ ]:
#output
print (sentence1_tokenizer)

In [ ]:
# If I use 1 or more sequences in my tokenizer, the token_type_ids will be identical
# If I use pairs, it will be different, this is what has been used in the notebook above

output_tokenizer = tokenizer (['My name is Daudi', 'I love NLP'], padding = True)
print (output_tokenizer)

output_tokenizer_2 = tokenizer ('My name is Daudi', 'I love NLP')
print (output_tokenizer_2)

# I can use list of sequences in each pair
# This is critical because it highlights how even when you have many s1, s2 pairs, the id_types will still match
# It helps explain how we can use this for our raw_datasets which has 2 lists each having many sequences
output_tokenizer_3 = tokenizer (['My name is Daudi', 'I love NLP'],['I am trying to understand something', 'It is great!'])
print (output_tokenizer_3)

#using output_tokenizer as input failed
#i think the expectation is to have just one set of input ids and not 2 or more
# it is mentioned that token_type_ids may not be available in other models, eg distil-bert, i have to confirm this though

print (tokenizer.convert_ids_to_tokens (output_tokenizer_2['input_ids']))


print (tokenizer.convert_ids_to_tokens (output_tokenizer_3['input_ids'][0]))


In [ ]:
raw_datasets['train']['sentence1']
type (raw_datasets['train']['sentence1'])
len (raw_datasets['train']['sentence1'])

In [ ]:
## Tokenize the sequences as pairs instead of two different ones

## TOKENIZATION


from transformers import AutoTokenizer

checkpoint = 'bert-base-uncased'
tokenizer = AutoTokenizer.from_pretrained (checkpoint)

# This approach is not proper, since we need the sentences to be used as a pair, not separately
sentence_tokenizer = tokenizer (raw_datasets['train']['sentence1'], raw_datasets['train']['sentence2'] , padding = True, truncation = True, return_tensors = 'tf')

In [ ]:
sentence_tokenizer

In [ ]:
sentence_tokenizer['token_type_ids'][0]

In [ ]:
sentence_tokenizer['token_type_ids'][1]

Looks like the approach we were using is not efficient if we want to pre-process our entire dataset, i.e the train, test and validation set. The best alternative will be to use define a tokenizer function, then use `Datasets.map()` to tokenize the entire dataset.

An additional thing is that padding is now disabled,so the sequences will have varying length of input ids and we will use batch in our tokenization coz it will make things faster for us. I will also not return tensors coz they were not using them in the course materials.

Think of this mapping as adding columns of a dataframe by applying a particular function to the original data frame.. the result will include all of the columns (by the way I have never tried this, I just think it is possible)

In [ ]:
# Our mapping function
def tokenize_function(example):
    return tokenizer(example["sentence1"], example["sentence2"], truncation=True)

In [ ]:
# Now we have a dataset that is tokenized
tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)
tokenized_datasets

In [ ]:
print (tokenized_datasets['train'][0])

In [ ]:
print (tokenized_datasets['train']['input_ids'])

# These were input ids of the entire train dataset
print (len(tokenized_datasets['train']['input_ids']))

In [ ]:
# input ids of the first first pair
print (tokenized_datasets['train']['input_ids'][0])
print (len (tokenized_datasets['train']['input_ids'][0]))

# input ids of the second pair
print (tokenized_datasets['train']['input_ids'][1])
print (len (tokenized_datasets['train']['input_ids'][1]))

In [ ]:
# This block takes the first 8 sentence-pairs in the tokenized train dataset
# It creates a dictionary containing just label and the outputs of the tokenizer
# The above columns are the ones that will be needed in training
# It creates a list of input ids and checks its length (using list comp)
# The result is the length of each sentense-pair input ids
# The input ids all have varying length, min is 32, max is 67

samples = tokenized_datasets["train"][:8]
samples = {k: v for k, v in samples.items() if k not in ["idx", "sentence1", "sentence2"]}
[len(x) for x in samples["input_ids"]]

In [ ]:
# Looking at components of a complex code help me break it down and understand
print (tokenized_datasets["train"][0])
print (samples)
print (samples.keys())

In [ ]:
from transformers import DataCollatorWithPadding

#tokenizer is an input that helps the collator to know details about padding
# details are such as which padding to use & whether to pad from left or right
# different tokenizers come with different such details
# we also return the output of the Collation as a tensor

data_collator = DataCollatorWithPadding(tokenizer = tokenizer, return_tensors='tf')

batch = data_collator (samples)

In [ ]:
type (batch) # looks like it is a dictionary
batch.keys() # keys are the same

batch['input_ids'].shape # 8 tokenized sentence pairs, with 67 input id values

In [ ]:
# looks like our tokenizer by default was padding wrt the longest sequence.. 67
 {k: v.shape for k,v in batch.items()}

In [ ]:
# We can collate the train, test and validation tokenized datasets independently
# In each set, we batch the set, and padding is done
# Batching is done according to the longest sequence

tokenized_train = tokenized_datasets['train'][:]
tokenized_train = {k:v for k,v in tokenized_train.items() if k not in (['sentence1', 'sentence2', 'idx'])}

tokenized_test = tokenized_datasets['test'][:]
tokenized_test = {k:v for k,v in tokenized_test.items() if k not in (['sentence1', 'sentence2', 'idx'])}

tokenized_validation = tokenized_datasets['validation'][:]
tokenized_validation = {k:v for k,v in tokenized_validation.items() if k not in (['sentence1', 'sentence2', 'idx'])}

train_collator = data_collator (tokenized_train)
test_collator = data_collator (tokenized_test)
validation_collator = data_collator (tokenized_validation)

train_batch = {k:v.shape for k,v in train_collator.items()}
test_batch = {k:v.shape for k,v in test_collator.items()}
validation_batch = {k:v.shape for k,v in validation_collator.items()}

# validation batch has the shortest padding
# test and train are almost the same
print (train_batch)
print (test_batch)
print (validation_batch)

In [ ]:
# to avoid all of the trouble of the previous cell, we can use this approach
# We leave out the test set because it will not be needed for training

tf_train_dataset = tokenized_datasets["train"].to_tf_dataset(
    columns=["attention_mask", "input_ids", "token_type_ids"],
    label_cols=["labels"],
    shuffle=True,
    collate_fn=data_collator,
    batch_size=8,
)

tf_validation_dataset = tokenized_datasets["validation"].to_tf_dataset(
    columns=["attention_mask", "input_ids", "token_type_ids"],
    label_cols=["labels"],
    shuffle=False,
    collate_fn=data_collator,
    batch_size=8,
)

In [ ]:
# Looks like this new format is not a dictionary..
# We'll confirm of its shape and everything in the next session

type (tf_train_dataset)
tf_train_dataset

## WEEK 9: FINE-TUNING A MODEL

### Recap:
In the previous lab, all I did was learn how to tokenize a training dataset that is downloaded from hugging face. The process involved tokenizing the dataset, but using the collator function for padding the train and validation sets separately. The notebooks show the process until that was achieved in the last cell.

__I think the same thing could have been archieved without even using the collator function. I have coded it below, but won't run until when I have more time__

In [ ]:
# Our mapping function
def tokenize_function(example):
    return tokenizer(example["sentence1"], example["sentence2"], truncation=True, padding = True, return_tensors = 'tf')

training = raw_datasets['train'].map(tokenize_function)
validation = raw_datasets['train'].map(tokenize_function)

In [ ]:
# This has to be run everytime I want to use datasets from huggingface

!pip install datasets

In [ ]:
## What we have done so far in summarized in 1 cell

from datasets import load_dataset
from transformers import AutoTokenizer, DataCollatorWithPadding
import numpy as np

raw_datasets = load_dataset("glue", "mrpc")
checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)


def tokenize_function(example):
    return tokenizer(example["sentence1"], example["sentence2"], truncation=True)


tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)

data_collator = DataCollatorWithPadding(tokenizer=tokenizer, return_tensors="tf")

tf_train_dataset = tokenized_datasets["train"].to_tf_dataset(
    columns=["attention_mask", "input_ids", "token_type_ids"],
    label_cols=["labels"],
    shuffle=True,
    collate_fn=data_collator,
    batch_size=8,
)

tf_validation_dataset = tokenized_datasets["validation"].to_tf_dataset(
    columns=["attention_mask", "input_ids", "token_type_ids"],
    label_cols=["labels"],
    shuffle=False,
    collate_fn=data_collator,
    batch_size=8,
)

In [ ]:
# Get the bert model
# We need to use it for classifying into 2 labels only
# We retain the head for embedding, etc, and add a classification head
# I am not very much familiar with the transformers architecture so mouth is shut
# Initially we never had the num_labels parameter, I guess it's bcoz we are fine-tuning

from transformers import TFAutoModelForSequenceClassification

model = TFAutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

In [ ]:
# This model can be improved on by a couple of tweaks

from tensorflow.keras.losses import SparseCategoricalCrossentropy


# the model will give out logits in its final layer
# we specify this to the loss function
# alternatively, we could have just used loss = 'Sparse...'

model.compile(
    optimizer="adam",
    loss=SparseCategoricalCrossentropy(from_logits=True),
    metrics=["accuracy"],
)

model.fit(
    tf_train_dataset,
    validation_data=tf_validation_dataset,
)

In [ ]:
# THIS IS THE INITIAL CODE THAT DEVELOPED ERRORS IN THE COMPILATION OF THE MODEL

# We introducing a decaying learning rate
# The number of steps is not very clear, I will have to come back later

from tensorflow.keras.optimizers.schedules import PolynomialDecay

batch_size = 8
num_epochs = 3
# The number of training steps is the number of samples in the dataset, divided by the batch size then multiplied
# by the total number of epochs. Note that the tf_train_dataset here is a batched tf.data.Dataset,
# not the original Hugging Face Dataset, so its len() is already num_samples // batch_size.
num_train_steps = len(tf_train_dataset) * num_epochs
lr_scheduler = PolynomialDecay(
    initial_learning_rate=5e-5, end_learning_rate=0.0, decay_steps=num_train_steps
)


from tensorflow.keras.optimizers import Adam

opt = Adam(learning_rate=lr_scheduler)

In [ ]:
# SUGGESTION BY CHATGPT DID NOT WORK

# We introducing a decaying learning rate
# The number of steps is not very clear, I will have to come back later

from tensorflow.keras.optimizers.schedules import PolynomialDecay

batch_size = 8
num_epochs = 3

# The number of training steps is the number of samples in the dataset, divided by the batch size then multiplied
# by the total number of epochs. Note that the tf_train_dataset here is a batched tf.data.Dataset,
# not the original Hugging Face Dataset, so its len() is already num_samples // batch_size.
num_train_steps = len(tf_train_dataset) * num_epochs
lr_scheduler = PolynomialDecay(
    initial_learning_rate=5e-5, end_learning_rate=0.0, decay_steps=num_train_steps
)

import tensorflow as tf
from tensorflow.keras.optimizers import Adam

opt = tf.keras.optimizers.legacy.Adam(learning_rate=lr_scheduler)

In [ ]:
# To retrain our model we initialize it again

from transformers import TFAutoModelForSequenceClassification
model = TFAutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

model.compile(
    optimizer=opt,
    loss=SparseCategoricalCrossentropy(from_logits=True),
    metrics=["accuracy"],
)

model.fit(
    tf_train_dataset,
    validation_data=tf_validation_dataset, epochs = 3
)